[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/WCC-Engineering/ENGR240/blob/main/Class%20Demos%20and%20Activities/Week%202/worksheet2_solution.ipynb)

# Worksheet 2.2: Flow Regime Analysis (SOLUTION)

## ENGR& 240: Engineering Computations
### Introduction to Scientific Computing with Python

## Objectives
- Practice implementing conditional statements in Python
- Develop functions with decision logic
- Use lambda functions for simple formula evaluations
- Visualize the relationship between different engineering variables

## Instructions

In this worksheet, we will analyze fluid flow in pipes, focusing on determining flow regimes and calculating friction factors. These calculations are fundamental in fluid mechanics and are used extensively in designing pipe systems, analyzing pressure drops, and understanding fluid behavior.

We'll work with the Reynolds number, a dimensionless quantity that helps predict whether fluid flow will be laminar or turbulent. Based on the flow regime, we'll implement different equations for calculating the friction factor, which is used to determine pressure drop in pipe systems.

## Background: Reynolds Number and Flow Regimes

The Reynolds number (Re) is a dimensionless quantity that predicts flow patterns in fluid mechanics. It is defined as:

$$Re = \frac{\rho v D}{\mu} = \frac{v D}{\nu}$$

Where:
- $\rho$ is the fluid density (kg/m³)
- $v$ is the flow velocity (m/s)
- $D$ is the pipe diameter (m)
- $\mu$ is the dynamic viscosity (kg/(m·s))
- $\nu$ is the kinematic viscosity (m²/s), where $\nu = \mu/\rho$

Based on the Reynolds number, flow can be categorized into three regimes:
1. **Laminar flow**: Re < 2300
2. **Transitional flow**: 2300 ≤ Re ≤ 4000
3. **Turbulent flow**: Re > 4000

### Friction Factor Calculations

The Darcy friction factor (f) is used to calculate pressure drop in pipe flow and depends on the flow regime:

1. **Laminar flow**:
   $$f = \frac{64}{Re}$$

2. **Transitional flow**: Typically interpolated between laminar and turbulent values

3. **Turbulent flow**: Several equations exist, including the Colebrook equation and its approximations. For smooth pipes, the Blasius equation is often used:
   $$f = \frac{0.316}{Re^{0.25}}$$

For rough pipes in turbulent flow, we use the Haaland equation (an approximation of the Colebrook equation):
$$f = \left(-1.8 \log_{10}\left[\frac{6.9}{Re} + \left(\frac{\epsilon/D}{3.7}\right)^{1.11}\right]\right)^{-2}$$

Where:
- $\epsilon$ is the pipe roughness (m)
- $D$ is the pipe diameter (m)
- $\epsilon/D$ is the relative roughness

## Part 1: Reynolds Number Calculation

First, we'll write a function to calculate the Reynolds number. Let's import the necessary libraries and define our function.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def calculate_reynolds(velocity, diameter, kinematic_viscosity):
    """
    Calculate the Reynolds number for pipe flow.
    
    Parameters:
    ----------
    velocity : float or array
        Flow velocity in m/s
    diameter : float or array
        Pipe diameter in m
    kinematic_viscosity : float or array
        Kinematic viscosity in m²/s
        
    Returns:
    -------
    reynolds : float or array
        Reynolds number (dimensionless)
    """
    reynolds = velocity * diameter / kinematic_viscosity
    return reynolds

# Test the function
# Example: Water at 20°C (kinematic viscosity ≈ 1.004e-6 m²/s)
# Pipe diameter: 0.05 m (50 mm)
# Flow velocity: 1 m/s
water_viscosity = 1.004e-6  # m²/s
pipe_diameter = 0.05  # m
flow_velocity = 1.0  # m/s

re = calculate_reynolds(flow_velocity, pipe_diameter, water_viscosity)
print(f"Reynolds number: {re:.0f}")

## Part 2: Classifying Flow Regimes

Now, let's write a function that classifies the flow regime based on the Reynolds number.

In [ ]:
def classify_flow_regime(reynolds):
    """
    Classify the flow regime based on the Reynolds number.
    
    Parameters:
    ----------
    reynolds : float or array
        Reynolds number
        
    Returns:
    -------
    regime : str or array of str
        Flow regime ('laminar', 'transitional', or 'turbulent')
    """
    if reynolds < 2300:
        regime = 'laminar'
    elif reynolds <= 4000:
        regime = 'transitional'
    else:
        regime = 'turbulent'
    
    return regime

# Test the function with our previous example
flow_type = classify_flow_regime(re)
print(f"Flow regime: {flow_type}")

Let's test our functions with a few different flow velocities to see how the Reynolds number and flow regime change:

In [ ]:
# Calculate the Reynolds number and flow regime for the different velocities
velocities = [0.01, 0.05, 3.0]  # m/s

for velocity in velocities:
    reynolds = calculate_reynolds(velocity, pipe_diameter, water_viscosity)
    regime = classify_flow_regime(reynolds)
    print(f"Velocity: {velocity} m/s, Reynolds: {reynolds:.0f}, Regime: {regime}")

## Part 3: Friction Factor Calculation

Next, we'll implement a function to calculate the Darcy friction factor based on the flow regime and pipe properties. This function will use conditional statements to apply the appropriate equation for each flow regime.

For this implementation, we'll use:
- The theoretical equation (64/Re) for laminar flow
- Linear interpolation between laminar and turbulent values for transitional flow
- The Haaland equation for turbulent flow

In [ ]:
def calculate_friction_factor(reynolds, relative_roughness=0):
    """
    Calculate the Darcy friction factor based on Reynolds number and relative roughness.
    
    Parameters:
    ----------
    reynolds : float or array
        Reynolds number
    relative_roughness : float or array, optional
        Relative pipe roughness (ε/D), dimensionless
        Default is 0 (smooth pipe)
        
    Returns:
    -------
    friction_factor : float or array
        Darcy friction factor
    """
    # Define the friction factor equations as lambda functions
    # This is a good use case for lambda functions - simple one-line formulas
    laminar_f = lambda re: 64 / re
    
    # Blasius equation for turbulent flow in smooth pipes
    blasius_f = lambda re: 0.316 / (re ** 0.25)
    
    # Haaland equation for turbulent flow in rough pipes
    haaland_f = lambda re, roughness: (-1.8 * np.log10((6.9/re) + (roughness/3.7)**1.11))**(-2)
    
    # Implement conditional logic based on flow regime
    # For the transitional regime, we'll use linear interpolation between the laminar and turbulent values
    # Linear interpolation formula: y = y1 + (x - x1) * (y2 - y1) / (x2 - x1)
    # Where: x1 = 2300, x2 = 4000, y1 = f_laminar, y2 = f_turbulent
    
    if reynolds < 2300:  # Laminar flow
        friction_factor = laminar_f(reynolds)
    elif reynolds <= 4000:  # Transitional flow
        # Linear interpolation between laminar and turbulent
        f_laminar = laminar_f(2300)
        f_turbulent = blasius_f(4000) if relative_roughness == 0 else haaland_f(4000, relative_roughness)
        
        # Linear interpolation factor
        t = (reynolds - 2300) / (4000 - 2300)
        friction_factor = f_laminar + t * (f_turbulent - f_laminar)
    else:  # Turbulent flow
        if relative_roughness == 0:
            # Smooth pipe - use Blasius equation
            friction_factor = blasius_f(reynolds)
        else:
            # Rough pipe - use Haaland equation
            friction_factor = haaland_f(reynolds, relative_roughness)
    
    return friction_factor

# Test the function for our previous example with a typical roughness
relative_roughness = 0.0001  # A typical value for commercial steel pipes
f = calculate_friction_factor(re, relative_roughness)
print(f"Friction factor: {f:.6f}")

## Part 4: Pressure Drop Calculation

Now that we can calculate the friction factor, let's create a function to calculate the pressure drop in a pipe. The pressure drop is given by the Darcy-Weisbach equation:

$$\Delta P = f \cdot \frac{L}{D} \cdot \frac{\rho v^2}{2}$$

Where:
- $\Delta P$ is the pressure drop (Pa)
- $f$ is the Darcy friction factor (dimensionless)
- $L$ is the pipe length (m)
- $D$ is the pipe diameter (m)
- $\rho$ is the fluid density (kg/m³)
- $v$ is the flow velocity (m/s)

In [ ]:
def calculate_pressure_drop(friction_factor, length, diameter, density, velocity):
    """
    Calculate pressure drop in a pipe using the Darcy-Weisbach equation.
    
    Parameters:
    ----------
    friction_factor : float or array
        Darcy friction factor
    length : float or array
        Pipe length in m
    diameter : float or array
        Pipe diameter in m
    density : float or array
        Fluid density in kg/m³
    velocity : float or array
        Flow velocity in m/s
        
    Returns:
    -------
    pressure_drop : float or array
        Pressure drop in Pa
    """
    pressure_drop = friction_factor * (length / diameter) * (density * velocity**2 / 2)
    return pressure_drop

# Test the function
water_density = 998.2  # kg/m³ at 20°C
pipe_length = 100  # m

pressure_drop = calculate_pressure_drop(f, pipe_length, pipe_diameter, water_density, flow_velocity)
print(f"Pressure drop: {pressure_drop:.2f} Pa ({pressure_drop/1000:.4f} kPa)")

## Part 5: Visualizing the Relationship Between Velocity and Pressure Drop

Finally, let's visualize the relationship between flow velocity and pressure drop. We'll create a plot that shows how the pressure drop changes with velocity, highlighting the transition between flow regimes.

In [ ]:
# Create a range of velocities that includes transition points
velocities = np.linspace(0.01, 3.0, 500)  # Using more points for smoother transition

# Calculate the transition velocities based on Reynolds number
re_2300_velocity = 2300 * water_viscosity / pipe_diameter  # Around 0.046 m/s
re_4000_velocity = 4000 * water_viscosity / pipe_diameter  # Around 0.080 m/s

# Print these transition velocities for reference
print(f"Laminar-transitional boundary velocity: {re_2300_velocity:.3f} m/s")
print(f"Transitional-turbulent boundary velocity: {re_4000_velocity:.3f} m/s")

# Initialize arrays to store results
reynolds_numbers = np.zeros_like(velocities)
regimes = np.full(len(velocities), '', dtype=object)  # Better initialization for string arrays
friction_factors = np.zeros_like(velocities)
pressure_drops = np.zeros_like(velocities)

# Calculate the results for each velocity
for i, v in enumerate(velocities):
    # Calculate Reynolds number
    reynolds = calculate_reynolds(v, pipe_diameter, water_viscosity)
    reynolds_numbers[i] = reynolds
    
    # Determine flow regime
    regime = classify_flow_regime(reynolds)
    regimes[i] = regime
    
    # Calculate friction factor
    f = calculate_friction_factor(reynolds, relative_roughness)
    friction_factors[i] = f
    
    # Calculate pressure drop
    p_drop = calculate_pressure_drop(f, pipe_length, pipe_diameter, water_density, v)
    pressure_drops[i] = p_drop / 1000  # Convert to kPa for better visualization

# Verify how many points fall into each regime
laminar_count = np.sum(regimes == 'laminar')
transitional_count = np.sum(regimes == 'transitional')
turbulent_count = np.sum(regimes == 'turbulent')
print(f"Points in each regime: Laminar: {laminar_count}, Transitional: {transitional_count}, Turbulent: {turbulent_count}")

# Identify indices for each flow regime for separate plotting
laminar_indices = np.where(regimes == 'laminar')[0]
transitional_indices = np.where(regimes == 'transitional')[0]
turbulent_indices = np.where(regimes == 'turbulent')[0]

# Create the plot
plt.figure(figsize=(12, 8))

# Plot each flow regime with a different color
if len(laminar_indices) > 0:
    plt.plot(velocities[laminar_indices], pressure_drops[laminar_indices], 'b-', 
             linewidth=2, label='Laminar Flow')
    
if len(transitional_indices) > 0:
    plt.plot(velocities[transitional_indices], pressure_drops[transitional_indices], 'g-', 
             linewidth=2, label='Transitional Flow')
    
if len(turbulent_indices) > 0:
    plt.plot(velocities[turbulent_indices], pressure_drops[turbulent_indices], 'r-', 
             linewidth=2, label='Turbulent Flow')

# Add regime transition lines
plt.axvline(x=re_2300_velocity, color='gray', linestyle='--', label='Re = 2300')
plt.axvline(x=re_4000_velocity, color='gray', linestyle=':', label='Re = 4000')

# Add annotations
plt.text(re_2300_velocity - 0.05, np.max(pressure_drops) * 0.9, 'Laminar-Transitional\nBoundary', 
         horizontalalignment='right', verticalalignment='top')
plt.text(re_4000_velocity + 0.05, np.max(pressure_drops) * 0.9, 'Transitional-Turbulent\nBoundary', 
         horizontalalignment='left', verticalalignment='top')

# Set log scale for y-axis since pressure drop varies greatly
plt.yscale('log')

# Add labels and title
plt.xlabel('Flow Velocity (m/s)')
plt.ylabel('Pressure Drop (kPa)')
plt.title(f'Pressure Drop vs. Flow Velocity for Different Flow Regimes\nPipe Diameter: {pipe_diameter*1000:.0f} mm, Relative Roughness: {relative_roughness}')
plt.grid(True, which='both', linestyle='--', alpha=0.7)
plt.legend(loc='lower right')
plt.tight_layout()
plt.show()

# Create another plot showing the friction factor vs. Reynolds number
plt.figure(figsize=(12, 6))

# Plot on a log-log scale
if len(laminar_indices) > 0:
    plt.loglog(reynolds_numbers[laminar_indices], friction_factors[laminar_indices], 'b-', 
             linewidth=2, label='Laminar Flow')
    
if len(transitional_indices) > 0:
    plt.loglog(reynolds_numbers[transitional_indices], friction_factors[transitional_indices], 'g-', 
             linewidth=2, label='Transitional Flow')
    
if len(turbulent_indices) > 0:
    plt.loglog(reynolds_numbers[turbulent_indices], friction_factors[turbulent_indices], 'r-', 
             linewidth=2, label='Turbulent Flow')

# Add vertical lines for regime transitions
plt.axvline(x=2300, color='gray', linestyle='--')
plt.axvline(x=4000, color='gray', linestyle=':')

# Add labels and title
plt.xlabel('Reynolds Number')
plt.ylabel('Friction Factor')
plt.title(f'Friction Factor vs. Reynolds Number for Different Flow Regimes\nRelative Roughness: {relative_roughness}')
plt.grid(True, which='both', linestyle='--', alpha=0.7)
plt.legend()
plt.tight_layout()
plt.show()

## Reflection

After completing this worksheet, answer the following questions:

1. How do conditional statements help you model different physical phenomena in engineering problems?

   **Answer**: Conditional statements allow us to implement different mathematical models or equations for different regimes or scenarios in engineering problems. In fluid dynamics, as seen in this worksheet, different physical equations apply to different flow regimes (laminar, transitional, turbulent). Conditional statements let us select the appropriate model based on calculated parameters like the Reynolds number. This approach mirrors how engineers approach real-world problems, where different physical laws apply under different conditions.

2. What are the benefits of breaking down a complex analysis into separate functions vs. having one large function that does everything?

   **Answer**: Breaking down complex analysis into separate functions offers numerous benefits:
   - **Modularity**: Each function handles a specific task, making the code easier to understand and maintain
   - **Reusability**: Individual functions can be reused in other analyses or projects
   - **Testability**: Smaller functions are easier to test individually, ensuring correctness
   - **Readability**: Code becomes more self-documenting when function names clearly describe their purpose
   - **Collaboration**: Different team members can work on different functions simultaneously
   - **Debugging**: Errors are easier to locate and fix in smaller functions
   
   In our pipe flow analysis, breaking it down into reynolds number calculation, flow regime classification, friction factor calculation, and pressure drop calculation made the code much more manageable and aligned with how engineers conceptually approach the problem.

3. How would you modify the code to handle a different fluid or pipe material? What changes would be needed?

   **Answer**: Our modular design makes these modifications straightforward:
   - For a different fluid, we would just need to change the input parameters: `kinematic_viscosity` and `density` to match the new fluid properties
   - For a different pipe material, we would need to use the appropriate relative roughness value for that material
   
   No changes to the core functions would be needed. For example, to analyze air flow through a steel pipe, we would simply update the input parameters:
   ```python
   air_viscosity = 1.48e-5  # m²/s at 20°C
   air_density = 1.2  # kg/m³ at 20°C
   steel_pipe_roughness = 0.00092  # relative roughness for a 50mm steel pipe
   
   # Then use these values in our calculations
   reynolds = calculate_reynolds(velocity, pipe_diameter, air_viscosity)
   friction = calculate_friction_factor(reynolds, steel_pipe_roughness)
   ```
   This flexibility demonstrates the power of our modular design and parameterized functions.

4. How did using lambda functions for mathematical formulas improve (or not improve) your code's readability and maintainability?

   **Answer**: Lambda functions significantly improved the code's readability and maintainability by:
   - Keeping mathematical formulas concise and in a format that closely resembles their mathematical notation
   - Separating the formula definitions from the conditional logic that selects which formula to use
   - Making the code more self-documenting, as each lambda function clearly represents a specific equation (e.g., laminar_f, blasius_f, haaland_f)
   
   The one-line nature of lambda functions is perfectly suited for these relatively simple mathematical formulas. For more complex calculations with multiple steps, regular named functions would be more appropriate. In this specific case, lambda functions provided an elegant solution that improved code organization without sacrificing clarity.